In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(2019)

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4,), (0.4,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False, num_workers=2)

In [3]:
import torch.nn as nn 
import torch.nn.functional as F

In [4]:
class myNet(nn.Module):
    def __init__(self):
        super(myNet, self).__init__()
        self.conv1=nn.Conv2d(1,400,3)
        self.pool1=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(400,400,3)
        self.pool2=nn.MaxPool2d(2,2)
        self.fc1=nn.Linear(400*5*5, 200)
        self.fc2=nn.Linear(200, 10)
        
    def forward(self,x):
        x=self.pool1(F.relu(self.conv1(x)))
        x=self.pool2(F.relu(self.conv2(x)))
        x=x.view(-1,400*5*5)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x
    

In [5]:
import torch.optim as optim

In [6]:
use_cuda = True
device = torch.device("cuda:0" if use_cuda else "cpu")

net = myNet()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
import time
for epoch in range(3):
    running_loss = 0
    t= time.time()
    for i, dl in enumerate(trainloader):
        opt.zero_grad()
        data, l = dl
        data, l = data.to(device), l.to(device)
        
        out = net(data)
        loss = criterion(out, l)
        loss.backward()
        opt.step()
        
        running_loss += loss.item()
        if i % 125 == 124:    # print every 2000 mini-batches
            print('%.4f'%(time.time()-t))
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            t=time.time()
            
print('Finished Training')

5.8547
[1,   125] loss: 0.087
3.4729
[1,   250] loss: 0.023
3.5976
[1,   375] loss: 0.017
3.4586
[1,   500] loss: 0.013
3.5181
[1,   625] loss: 0.011
3.5566
[1,   750] loss: 0.009
3.5309
[1,   875] loss: 0.009
4.2727
[2,   125] loss: 0.007
3.6456
[2,   250] loss: 0.007
3.6682
[2,   375] loss: 0.006
3.5741
[2,   500] loss: 0.006
3.4541
[2,   625] loss: 0.006
3.4601
[2,   750] loss: 0.005
3.6205
[2,   875] loss: 0.005
4.0802
[3,   125] loss: 0.004
3.6182
[3,   250] loss: 0.005
3.5713
[3,   375] loss: 0.005
3.4895
[3,   500] loss: 0.004
3.4827
[3,   625] loss: 0.004
3.4803
[3,   750] loss: 0.005
3.5696
[3,   875] loss: 0.004
Finished Training


In [8]:
correct = 0
total = 0
with torch.no_grad():
    for i,data in enumerate(testloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        con = predicted != labels
#         for im, p in zip(images[con], predicted[con]):
#             print(p)
#             plt.imshow(im.cpu().reshape(28,28))
#             plt.show()

print('Accuracy of the network on the 10000 test images: %.3f %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98.070 %
